## types of recommendation system :

### content based recommendation system: recommends movies to a user based on the content of the films that they have watched before
### popularity based recommendation system : recommends which movies or series has been very popular 
### collaborative recommendation system : groups people based on their watching pattern , recommend movies based on the watching pattern of an individual based on some previous data that it has collected from the other individuals , based on user's behavior 

In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

**Data Collection and Pre-Processing**

In [3]:
# loading the data from a csv file to a pandas dataframe
movies_data = pd.read_csv('movies.csv')

In [4]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
# number of rows and columns in the dataframe
movies_data.shape

(4803, 24)

In [6]:
# selecting the relevent features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [7]:
# replacing the null values with null string (empty string)
for feature in selected_features:
    movies_data[feature]=movies_data[feature].fillna('')

In [8]:
# combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [9]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
# converting the text data to feature vectors (numeric values)
vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

**Cosine Similarity**

In [12]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [13]:
# getting the movie name from the user
movie_name = input('Enter your favourite movie name :')

Enter your favourite movie name :seven pounds


In [14]:
# creating a list with all the movie names given in the dataset 

list_of_all_titles = movies_data['title'].tolist()

In [15]:
# findig the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)

In [18]:
close_match = find_close_match[0]
close_match

'Seven Pounds'

In [21]:
# finding the index of the movie with title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
index_of_the_movie

865

In [22]:
# getting a list of similar movies 
similarity_score = list(enumerate(similarity[index_of_the_movie]))
similarity_score

[(0, 0.0),
 (1, 0.01071747679561559),
 (2, 0.03145777604224594),
 (3, 0.0339676009275551),
 (4, 0.0),
 (5, 0.012720486525890478),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0095315311732522),
 (9, 0.0),
 (10, 0.0),
 (11, 0.032810713751335735),
 (12, 0.0),
 (13, 0.0),
 (14, 0.023341550215103327),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.040427950479450754),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.01233623841614619),
 (24, 0.0025715265546010116),
 (25, 0.0027444330738967144),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.019321761547580266),
 (30, 0.012375777288706098),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0097173939287528),
 (36, 0.008756147395458342),
 (37, 0.0),
 (38, 0.0),
 (39, 0.017391801032959123),
 (40, 0.009777751287883158),
 (41, 0.01374296708422156),
 (42, 0.009221105188915512),
 (43, 0.0),
 (44, 0.0),
 (45, 0.002472707542381335),
 (46, 0.008870781460236258),
 (47, 0.015795298125294777),
 (48, 0.0),
 (49, 0.0022809817071526763),
 (50, 0.0),
 (51, 0.0),
 (

In [23]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similar_movies

[(865, 1.0000000000000002),
 (1553, 0.1758160820512676),
 (824, 0.16771747848575408),
 (3415, 0.1495712749832424),
 (4585, 0.12970810914283698),
 (3575, 0.12610056157681165),
 (2921, 0.11679074855999359),
 (3869, 0.11457087686600256),
 (1475, 0.11117469892211068),
 (447, 0.11098449052437503),
 (505, 0.1042184043295166),
 (3487, 0.10264803235389568),
 (1773, 0.10225597731707364),
 (2764, 0.09777448516990991),
 (4217, 0.08888505347997691),
 (1979, 0.08693180421654381),
 (1017, 0.08590784175945766),
 (3819, 0.08344747552425479),
 (183, 0.08324721002697363),
 (1963, 0.08232766878752544),
 (370, 0.08174677361840195),
 (847, 0.08141855421488342),
 (1235, 0.08116266509593204),
 (102, 0.08052221407949127),
 (669, 0.08022650943669037),
 (2495, 0.07983921025469128),
 (3631, 0.07970758713800072),
 (3699, 0.07910025047271271),
 (804, 0.07870915427137408),
 (3436, 0.07748002394626063),
 (311, 0.07705183795191842),
 (3502, 0.07621050983219785),
 (1384, 0.07525825133532763),
 (460, 0.0747588750141190

In [24]:
# print the name of similar movies based on the index 

print('Movies suggested for you : \n')

i=1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if(i<30):
        print(i,'.',title_from_index)
        i+=1

Movies suggested for you : 

1 . Seven Pounds
2 . Se7en
3 . The Pursuit of Happyness
4 . Remember Me, My Love
5 . The Beyond
6 . Clerks II
7 . Drive Hard
8 . The Dead Girl
9 . Playing for Keeps
10 . Eagle Eye
11 . The League of Extraordinary Gentlemen
12 . Shattered Glass
13 . Mindhunters
14 . Light It Up
15 . Kids
16 . Play It to the Bone
17 . Bedazzled
18 . Defendor
19 . The Hunger Games: Catching Fire
20 . Think Like a Man Too
21 . Now You See Me 2
22 . Semi-Pro
23 . Rent
24 . The Hunger Games: Mockingjay - Part 2
25 . Sin City: A Dame to Kill For
26 . Trance
27 . Heavenly Creatures
28 . 25th Hour
29 . After the Sunset
